In [3]:
import sys
import numpy
import pandas
import sklearn
import keras

print('Python: {}'.format(sys.version))
print('numpy: {}'.format(numpy.__version__))
print('pandas: {}'.format(pandas.__version__))
print('sklearn: {}'.format(sklearn.__version__))
print('Keras: {}'.format(keras.__version__))

Using TensorFlow backend.


Python: 3.6.9 (default, Apr 18 2020, 01:56:04) 
[GCC 8.4.0]
numpy: 1.18.5
pandas: 1.0.5
sklearn: 0.22.2.post1
Keras: 2.3.1


In [4]:
!wget https://raw.githubusercontent.com/rrichajalota/Pima-Indians-Diabetes-kaggle/master/diabetes.csv

--2020-06-23 16:12:46--  https://raw.githubusercontent.com/rrichajalota/Pima-Indians-Diabetes-kaggle/master/diabetes.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23873 (23K) [text/plain]
Saving to: ‘diabetes.csv’

diabetes.csv        100%[===================>]  23.31K  --.-KB/s    in 0.01s   

2020-06-23 16:12:46 (1.90 MB/s) - ‘diabetes.csv’ saved [23873/23873]



In [5]:
import pandas as pd
import numpy as np

df = pd.read_csv('diabetes.csv')

In [6]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [7]:
df[df['Glucose'] == 0]

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
75,1,0,48,20,0,24.7,0.140,22,0
182,1,0,74,20,23,27.7,0.299,21,0
342,1,0,68,35,0,32.0,0.389,22,0
349,5,0,80,32,0,41.0,0.346,37,1
502,6,0,68,41,0,39.0,0.727,41,1


In [8]:
columns = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

for col in columns:
    df[col].replace(0, np.NaN, inplace=True)

df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,763.000000,733.000000,541.000000,394.000000,757.000000,768.000000,768.000000,768.000000
mean,3.845052,121.686763,72.405184,29.153420,155.548223,32.457464,0.471876,33.240885,0.348958
std,3.369578,30.535641,12.382158,10.476982,118.775855,6.924988,0.331329,11.760232,0.476951
min,0.000000,44.000000,24.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,64.000000,22.000000,76.250000,27.500000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,29.000000,125.000000,32.300000,0.372500,29.000000,0.000000
75%,6.000000,141.000000,80.000000,36.000000,190.000000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [9]:
df.dropna(inplace=True)

df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,3.301020,122.627551,70.663265,29.145408,156.056122,33.086224,0.523046,30.864796,0.331633
std,3.211424,30.860781,12.496092,10.516424,118.841690,7.027659,0.345488,10.200777,0.471401
min,0.000000,56.000000,24.000000,7.000000,14.000000,18.200000,0.085000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,21.000000,76.750000,28.400000,0.269750,23.000000,0.000000
50%,2.000000,119.000000,70.000000,29.000000,125.500000,33.200000,0.449500,27.000000,0.000000
75%,5.000000,143.000000,78.000000,37.000000,190.000000,37.100000,0.687000,36.000000,1.000000
max,17.000000,198.000000,110.000000,63.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [10]:
dataset = df.values
print(dataset.shape)

(392, 9)


In [11]:
X = dataset[:, 0:8]
Y = dataset[:, 8].astype(int)

In [12]:
print(X.shape)
print(Y.shape)
print(Y[:5])

(392, 8)
(392,)
[0 1 1 1 1]


In [13]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X)

In [14]:
print(scaler)

StandardScaler(copy=True, with_mean=True, with_std=True)


In [15]:
X_standardized = scaler.transform(X)

data = pd.DataFrame(X_standardized)
data.describe()

,0,1,2,3,4,5,6,7
count,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02
mean,-4.021726e-17,3.129583e-17,-4.641624e-16,1.042250e-16,6.485742e-17,1.543550e-16,3.880116e-17,1.028089e-16
std,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00
min,-1.029213e+00,-2.161731e+00,-3.739001e+00,-2.108484e+00,-1.196867e+00,-2.120941e+00,-1.269525e+00,-9.682991e-01
25%,-7.174265e-01,-7.665958e-01,-6.941640e-01,-7.755315e-01,-6.681786e-01,-6.676780e-01,-7.340909e-01,-7.719850e-01
50%,-4.056403e-01,-1.176959e-01,-5.314565e-02,-1.384444e-02,-2.574448e-01,1.621036e-02,-2.131475e-01,-3.793569e-01
75%,5.297185e-01,6.609841e-01,5.878727e-01,7.478426e-01,2.859877e-01,5.718696e-01,4.751644e-01,5.040564e-01
max,4.271153e+00,2.445459e+00,3.151946e+00,3.223325e+00,5.812990e+00,4.846172e+00,5.497667e+00,4.921123e+00


In [16]:
from sklearn.model_selection import GridSearchCV, KFold

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam

In [17]:
def create_model():
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

model = create_model()
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
seed = 6
np.random.seed(seed)

model = KerasClassifier(build_fn = create_model, verbose = 1)

batch_size = [10, 20, 40]
epochs = [10, 50, 100]

param_grid = dict(batch_size=batch_size, epochs=epochs)

grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] batch_size=10, epochs=10 ........................................
Epoch 1/10
313/313 [==============================] - 2s 6ms/step - loss: 0.6129 - accuracy: 0.6869
Epoch 2/10
313/313 [==============================] - 0s 267us/step - loss: 0.4633 - accuracy: 0.7380
Epoch 3/10
313/313 [==============================] - 0s 263us/step - loss: 0.4431 - accuracy: 0.7732
Epoch 4/10
313/313 [==============================] - 0s 260us/step - loss: 0.4254 - accuracy: 0.8019
Epoch 5/10
313/313 [==============================] - 0s 309us/step - loss: 0.4275 - accuracy: 0.7987
Epoch 6/10
313/313 [==============================] - 0s 265us/step - loss: 0.4180 - accuracy: 0.7955
Epoch 7/10
313/313 [==============================] - 0s 264us/step - loss: 0.4140 - accuracy: 0.7955
Epoch 8/10
313/313 [==============================] - 0s 266us/step - loss: 0.4122 - accuracy: 0.8083
Epoch 9/10
313/313 [==============================] - 0

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.1s remaining:    0.0s


Epoch 1/10
313/313 [==============================] - 0s 535us/step - loss: 0.5849 - accuracy: 0.6773
Epoch 2/10
313/313 [==============================] - 0s 280us/step - loss: 0.4262 - accuracy: 0.8115
Epoch 3/10
313/313 [==============================] - 0s 271us/step - loss: 0.4004 - accuracy: 0.8147
Epoch 4/10
313/313 [==============================] - 0s 300us/step - loss: 0.3847 - accuracy: 0.8307
Epoch 5/10
313/313 [==============================] - 0s 258us/step - loss: 0.3759 - accuracy: 0.8275
Epoch 6/10
313/313 [==============================] - 0s 282us/step - loss: 0.3721 - accuracy: 0.8307
Epoch 7/10
313/313 [==============================] - 0s 288us/step - loss: 0.3657 - accuracy: 0.8339
Epoch 8/10
313/313 [==============================] - 0s 256us/step - loss: 0.3736 - accuracy: 0.8275
Epoch 9/10
313/313 [==============================] - 0s 300us/step - loss: 0.3652 - accuracy: 0.8371
Epoch 10/10
79/79 [==============================] - 0s 323us/step
[CV] ..........

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    4.6s remaining:    0.0s


...

Epoch 1/50
314/314 [==============================] - 0s 531us/step - loss: 0.6273 - accuracy: 0.6688
Epoch 2/50
314/314 [==============================] - 0s 257us/step - loss: 0.4994 - accuracy: 0.7516
Epoch 3/50
314/314 [==============================] - 0s 282us/step - loss: 0.4705 - accuracy: 0.7739
Epoch 4/50
314/314 [==============================] - 0s 259us/step - loss: 0.4610 - accuracy: 0.7707
Epoch 5/50
314/314 [==============================] - 0s 274us/step - loss: 0.4468 - accuracy: 0.7771
Epoch 6/50
314/314 [==============================] - 0s 267us/step - loss: 0.4388 - accuracy: 0.7771
Epoch 7/50
314/314 [==============================] - 0s 289us/step - loss: 0.4306 - accuracy: 0.7803
Epoch 8/50
314/314 [==============================] - 0s 288us/step - loss: 0.4264 - accuracy: 0.7962
Epoch 9/50
314/314 [==============================] - 0s 271us/step - loss: 0.4182 - accuracy: 0.8089
Epoch 10/50
314/314 [==============================] - 0s 266us/step - loss: 0.415

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   29.2s remaining:    0.0s


Epoch 1/50
314/314 [==============================] - 0s 533us/step - loss: 0.6579 - accuracy: 0.6592
Epoch 2/50
314/314 [==============================] - 0s 249us/step - loss: 0.4919 - accuracy: 0.7548
Epoch 3/50
314/314 [==============================] - 0s 265us/step - loss: 0.4583 - accuracy: 0.7675
Epoch 4/50
314/314 [==============================] - 0s 252us/step - loss: 0.4474 - accuracy: 0.7643
Epoch 5/50
314/314 [==============================] - 0s 272us/step - loss: 0.4489 - accuracy: 0.7516
Epoch 6/50
314/314 [==============================] - 0s 279us/step - loss: 0.4418 - accuracy: 0.7707
Epoch 7/50
314/314 [==============================] - 0s 265us/step - loss: 0.4385 - accuracy: 0.7771
Epoch 8/50
314/314 [==============================] - 0s 258us/step - loss: 0.4349 - accuracy: 0.7771
Epoch 9/50
314/314 [==============================] - 0s 304us/step - loss: 0.4329 - accuracy: 0.7834
Epoch 10/50
314/314 [==============================] - 0s 285us/step - loss: 0.433

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  2.6min finished


Epoch 1/50
392/392 [==============================] - 0s 526us/step - loss: 0.6589 - accuracy: 0.7015
Epoch 2/50
392/392 [==============================] - 0s 317us/step - loss: 0.5848 - accuracy: 0.7781
Epoch 3/50
392/392 [==============================] - 0s 267us/step - loss: 0.5459 - accuracy: 0.7806
Epoch 4/50
392/392 [==============================] - 0s 263us/step - loss: 0.5115 - accuracy: 0.7934
Epoch 5/50
392/392 [==============================] - 0s 269us/step - loss: 0.4887 - accuracy: 0.7985
Epoch 6/50
392/392 [==============================] - 0s 292us/step - loss: 0.4724 - accuracy: 0.7985
Epoch 7/50
392/392 [==============================] - 0s 271us/step - loss: 0.4605 - accuracy: 0.7934
Epoch 8/50
392/392 [==============================] - 0s 264us/step - loss: 0.4475 - accuracy: 0.8138
Epoch 9/50
392/392 [==============================] - 0s 290us/step - loss: 0.4378 - accuracy: 0.8036
Epoch 10/50
392/392 [==============================] - 0s 281us/step - loss: 0.438

In [19]:
seed = 6
np.random.seed(seed)

model = KerasClassifier(build_fn = create_model, epochs = 100, batch_size = 20, verbose = 0)

learn_rate = [0.001, 0.01, 0.1]
dropout_rate = [0.0, 0.1, 0.2]

param_grid = dict(batch_size=batch_size, epochs=epochs)

grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] batch_size=10, epochs=10 ........................................
[CV] ............ batch_size=10, epochs=10, score=0.759, total=   1.4s
[CV] batch_size=10, epochs=10 ........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s remaining:    0.0s


[CV] ............ batch_size=10, epochs=10, score=0.633, total=   1.4s
[CV] batch_size=10, epochs=10 ........................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.8s remaining:    0.0s


...

[CV] ............ batch_size=10, epochs=50, score=0.846, total=   4.3s
[CV] batch_size=10, epochs=50 ........................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   24.3s remaining:    0.0s


[CV] ............ batch_size=10, epochs=50, score=0.833, total=   4.5s
[CV] batch_size=10, epochs=100 .......................................
[CV] ........... batch_size=10, epochs=100, score=0.759, total=   8.8s
[CV] batch_size=10, epochs=100 .......................................
[CV] ........... batch_size=10, epochs=100, score=0.633, total=   8.4s
[CV] batch_size=10, epochs=100 .......................................
[CV] ........... batch_size=10, epochs=100, score=0.795, total=   8.6s
[CV] batch_size=10, epochs=100 .......................................
[CV] ........... batch_size=10, epochs=100, score=0.808, total=   8.5s
[CV] batch_size=10, epochs=100 .......................................
[CV] ........... batch_size=10, epochs=100, score=0.833, total=   8.2s
[CV] batch_size=20, epochs=10 ........................................
[CV] ............ batch_size=20, epochs=10, score=0.785, total=   1.0s
[CV] batch_size=20, epochs=10 ........................................
[CV] .

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  2.3min finished


Best: 0.7809477329254151, using {'batch_size': 40, 'epochs': 100}
0.7759168982505799 (0.07655375483984386) with: {'batch_size': 10, 'epochs': 10}
0.7708536148071289 (0.07279727677270621) with: {'batch_size': 10, 'epochs': 50}
0.7656604886054993 (0.07051232414656733) with: {'batch_size': 10, 'epochs': 100}
0.758000648021698 (0.08198364564387145) with: {'batch_size': 20, 'epochs': 10}
0.7453099608421325 (0.07270422737180122) with: {'batch_size': 20, 'epochs': 50}
0.758000648021698 (0.06554185882227492) with: {'batch_size': 20, 'epochs': 100}
0.7681921482086181 (0.07000468071355816) with: {'batch_size': 40, 'epochs': 10}
0.768321979045868 (0.08318150544397801) with: {'batch_size': 40, 'epochs': 50}
0.7809477329254151 (0.058060053163787005) with: {'batch_size': 40, 'epochs': 100}


In [23]:
from keras.layers import Dropout

seed = 6
np.random.seed(seed)

def create_model(learn_rate, dropout_rate):
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))

    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

model = KerasClassifier(build_fn = create_model, epochs = 100, batch_size = 20, verbose = 0)

learn_rate = [0.001, 0.01, 0.1]
dropout_rate = [0.0, 0.1, 0.2]

param_grid = dict(learn_rate=learn_rate, dropout_rate=dropout_rate)

grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................
[CV] .. dropout_rate=0.0, learn_rate=0.001, score=0.759, total=   4.3s
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.3s remaining:    0.0s


[CV] .. dropout_rate=0.0, learn_rate=0.001, score=0.658, total=   4.5s
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    8.8s remaining:    0.0s


...

[CV] ... dropout_rate=0.0, learn_rate=0.01, score=0.821, total=   4.5s
[CV] dropout_rate=0.0, learn_rate=0.01 ...............................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   40.8s remaining:    0.0s


[CV] ... dropout_rate=0.0, learn_rate=0.01, score=0.821, total=   4.7s
[CV] dropout_rate=0.0, learn_rate=0.1 ................................
[CV] .... dropout_rate=0.0, learn_rate=0.1, score=0.785, total=   4.4s
[CV] dropout_rate=0.0, learn_rate=0.1 ................................
[CV] .... dropout_rate=0.0, learn_rate=0.1, score=0.646, total=   4.4s
[CV] dropout_rate=0.0, learn_rate=0.1 ................................
[CV] .... dropout_rate=0.0, learn_rate=0.1, score=0.769, total=   5.0s
[CV] dropout_rate=0.0, learn_rate=0.1 ................................
[CV] .... dropout_rate=0.0, learn_rate=0.1, score=0.821, total=   4.9s
[CV] dropout_rate=0.0, learn_rate=0.1 ................................
[CV] .... dropout_rate=0.0, learn_rate=0.1, score=0.821, total=   4.7s
[CV] dropout_rate=0.1, learn_rate=0.001 ..............................
[CV] .. dropout_rate=0.1, learn_rate=0.001, score=0.772, total=   5.1s
[CV] dropout_rate=0.1, learn_rate=0.001 ..............................
[CV] .

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  3.8min finished


Best: 0.7962998986244202, using {'dropout_rate': 0.2, 'learn_rate': 0.1}
0.7732878804206849 (0.06229264746895163) with: {'dropout_rate': 0.0, 'learn_rate': 0.001}
0.7733203530311584 (0.05903306611422605) with: {'dropout_rate': 0.0, 'learn_rate': 0.01}
0.7681272387504577 (0.06448008440275914) with: {'dropout_rate': 0.0, 'learn_rate': 0.1}
0.7810775518417359 (0.0913292219631152) with: {'dropout_rate': 0.1, 'learn_rate': 0.001}
0.7708860754966735 (0.07947208927298932) with: {'dropout_rate': 0.1, 'learn_rate': 0.01}
0.7555014610290527 (0.0764455353064309) with: {'dropout_rate': 0.1, 'learn_rate': 0.1}
0.7912041544914246 (0.07460814525317082) with: {'dropout_rate': 0.2, 'learn_rate': 0.001}
0.7658552289009094 (0.09434682108574105) with: {'dropout_rate': 0.2, 'learn_rate': 0.01}
0.7962998986244202 (0.0734586667237363) with: {'dropout_rate': 0.2, 'learn_rate': 0.1}


In [24]:
seed = 6
np.random.seed(seed)

def create_model(activation, init):
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer = init, activation = activation))
    model.add(Dense(4, input_dim = 8, kernel_initializer = init, activation = activation))
    model.add(Dense(1, activation='sigmoid'))

    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

model = KerasClassifier(build_fn = create_model, epochs = 100, batch_size = 20, verbose = 0)

activation = ['softmax', 'relu', 'tanh', 'linear']
init = ['uniform', 'normal', 'zero']

param_grid = dict(activation = activation, init = init)

grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] activation=softmax, init=uniform ................................
[CV] .... activation=softmax, init=uniform, score=0.734, total=   5.2s
[CV] activation=softmax, init=uniform ................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.2s remaining:    0.0s


[CV] .... activation=softmax, init=uniform, score=0.696, total=   4.8s
[CV] activation=softmax, init=uniform ................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   10.0s remaining:    0.0s


...

[CV] ..... activation=softmax, init=normal, score=0.859, total=   5.0s
[CV] activation=softmax, init=normal .................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   43.5s remaining:    0.0s


[CV] ..... activation=softmax, init=normal, score=0.782, total=   4.8s
[CV] activation=softmax, init=zero ...................................
[CV] ....... activation=softmax, init=zero, score=0.646, total=   5.0s
[CV] activation=softmax, init=zero ...................................
[CV] ....... activation=softmax, init=zero, score=0.570, total=   4.7s
[CV] activation=softmax, init=zero ...................................
[CV] ....... activation=softmax, init=zero, score=0.705, total=   4.7s
[CV] activation=softmax, init=zero ...................................
[CV] ....... activation=softmax, init=zero, score=0.744, total=   4.8s
[CV] activation=softmax, init=zero ...................................
[CV] ....... activation=softmax, init=zero, score=0.679, total=   4.8s
[CV] activation=relu, init=uniform ...................................
[CV] ....... activation=relu, init=uniform, score=0.646, total=   4.6s
[CV] activation=relu, init=uniform ...................................
[CV] .

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:  4.8min finished


Best: 0.7936708807945252, using {'activation': 'linear', 'init': 'uniform'}
0.7604349255561829 (0.04235388257314441) with: {'activation': 'softmax', 'init': 'uniform'}
0.7656605005264282 (0.06443615376272542) with: {'activation': 'softmax', 'init': 'normal'}
0.6686789989471436 (0.05899773033245815) with: {'activation': 'softmax', 'init': 'zero'}
0.7529698133468627 (0.07032378878295327) with: {'activation': 'relu', 'init': 'uniform'}
0.783479380607605 (0.053883432261351304) with: {'activation': 'relu', 'init': 'normal'}
0.6686789989471436 (0.05899773033245815) with: {'activation': 'relu', 'init': 'zero'}
0.763063931465149 (0.06240231535084014) with: {'activation': 'tanh', 'init': 'uniform'}
0.758000648021698 (0.08513100762018529) with: {'activation': 'tanh', 'init': 'normal'}
0.6686789989471436 (0.05899773033245815) with: {'activation': 'tanh', 'init': 'zero'}
0.7936708807945252 (0.08079181748331918) with: {'activation': 'linear', 'init': 'uniform'}
0.7885426759719849 (0.078065516234129

In [25]:
seed = 6
np.random.seed(seed)

def create_model(neuron1, neuron2):
    model = Sequential()
    model.add(Dense(neuron1, input_dim = 8, kernel_initializer='normal', activation='linear'))
    model.add(Dense(neuron2, input_dim = 8, kernel_initializer='normal', activation='linear'))
    model.add(Dense(1, activation='sigmoid'))

    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

model = KerasClassifier(build_fn = create_model, epochs = 100, batch_size = 20, verbose = 0)

neuron1 = [4, 8, 16]
neuron2 = [2, 4, 8]

param_grid = dict(neuron1 = neuron1, neuron2 = neuron2)

grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] neuron1=4, neuron2=2 ............................................


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................ neuron1=4, neuron2=2, score=0.835, total=   5.0s
[CV] neuron1=4, neuron2=2 ............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.0s remaining:    0.0s


[CV] ................ neuron1=4, neuron2=2, score=0.608, total=   5.2s
[CV] neuron1=4, neuron2=2 ............................................


...

[CV] ................ neuron1=4, neuron2=4, score=0.846, total=   4.7s
[CV] neuron1=4, neuron2=4 ............................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   43.3s remaining:    0.0s


[CV] ................ neuron1=4, neuron2=4, score=0.833, total=   4.7s
[CV] neuron1=4, neuron2=8 ............................................
[CV] ................ neuron1=4, neuron2=8, score=0.835, total=   4.6s
[CV] neuron1=4, neuron2=8 ............................................
[CV] ................ neuron1=4, neuron2=8, score=0.633, total=   4.6s
[CV] neuron1=4, neuron2=8 ............................................
[CV] ................ neuron1=4, neuron2=8, score=0.821, total=   5.3s
[CV] neuron1=4, neuron2=8 ............................................
[CV] ................ neuron1=4, neuron2=8, score=0.846, total=   5.1s
[CV] neuron1=4, neuron2=8 ............................................
[CV] ................ neuron1=4, neuron2=8, score=0.833, total=   4.8s
[CV] neuron1=8, neuron2=2 ............................................
[CV] ................ neuron1=8, neuron2=2, score=0.835, total=   4.7s
[CV] neuron1=8, neuron2=2 ............................................
[CV] .

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  3.7min finished


Best: 0.7937033295631408, using {'neuron1': 16, 'neuron2': 8}
0.7860434889793396 (0.0897482447860841) with: {'neuron1': 4, 'neuron2': 2}
0.7936708807945252 (0.08079181748331918) with: {'neuron1': 4, 'neuron2': 4}
0.7936708807945252 (0.08079181748331918) with: {'neuron1': 4, 'neuron2': 8}
0.7885751247406005 (0.08587248740892742) with: {'neuron1': 8, 'neuron2': 2}
0.7911716818809509 (0.09262406801687142) with: {'neuron1': 8, 'neuron2': 4}
0.7886075854301453 (0.09087249991485369) with: {'neuron1': 8, 'neuron2': 8}
0.7911392331123352 (0.08583080767748588) with: {'neuron1': 16, 'neuron2': 2}
0.7860434889793396 (0.0897482447860841) with: {'neuron1': 16, 'neuron2': 4}
0.7937033295631408 (0.08760908345665606) with: {'neuron1': 16, 'neuron2': 8}


In [26]:
y_pred = grid.predict(X_standardized)

In [27]:
print(y_pred.shape)

(392, 1)


In [28]:
print(y_pred[:5])

[[0]
 [1]
 [0]
 [1]
 [1]]


In [29]:
from sklearn.metrics import classification_report, accuracy_score

print(accuracy_score(Y, y_pred))
print(classification_report(Y, y_pred))

0.7882653061224489
              precision    recall  f1-score   support

           0       0.81      0.90      0.85       262
           1       0.73      0.57      0.64       130

    accuracy                           0.79       392
   macro avg       0.77      0.73      0.75       392
weighted avg       0.78      0.79      0.78       392

